# Weekly Efficiency Analysis (2024)

**Goal**: Track player efficiency week-by-week to identify trends.

**Metrics**:
- **Weekly Performance**: Standard Fantasy Points (PPR) as a proxy for weekly value.
- **Weekly Cost**: `Cap Hit / 17`.
- **Weekly Efficiency**: `Fantasy Points / Weekly Cost ($M)`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option('display.max_columns', 50)
sns.set_style("whitegrid")

## 1. Load Data

In [ ]:
# Load Game Logs
LOGS_PATH = "../data/raw/pfr/game_logs_2024.csv"
logs = pd.read_csv(LOGS_PATH)

# Load Financials
SPOTRAC_PATH = "../data/raw/spotrac_player_rankings_2024_2026w05_20260130_183232.csv"
financials = pd.read_csv(SPOTRAC_PATH)

# Clean Financials
def clean_name(name):
    if pd.isna(name): return ""
    name = name.replace(" Jr.", "").replace(" Sr.", "").replace(" III", "").replace(" II", "")
    return name.strip().lower()

financials['clean_name'] = financials['player_name'].apply(clean_name)

# Parse Cap Hit
if 'cap_hit_millions' in financials.columns:
    financials['cap_hit_m'] = financials['cap_hit_millions']
elif 'value' in financials.columns:
    financials['cap_hit_m'] = financials['value'].replace('[\$,]', '', regex=True).astype(float) / 1_000_000
else:
    # Fallback to total contract value / length proxy if needed, but rankings usually has current value
    financials['cap_hit_m'] = financials['total_contract_value_millions'] / financials['contract_length_years']

# Weekly Cost
financials['weekly_cost_m'] = financials['cap_hit_m'] / 17

# Clean Logs
# PFR headers are messy: Unnamed: 0_level_0_Player -> Player
def clean_col(col):
    if 'Player' in col: return 'Player'
    if '_Tm' in col: return 'Team'
    return col

logs.columns = [clean_col(c) for c in logs.columns]

# Fill NaNs with 0 for stats
stat_cols = ['Passing_Yds', 'Passing_TD', 'Passing_Int', 
             'Rushing_Yds', 'Rushing_TD', 
             'Receiving_Rec', 'Receiving_Yds', 'Receiving_TD', 
             'Fumbles_FL']

for c in stat_cols:
    if c in logs.columns:
        logs[c] = logs[c].fillna(0)
    else:
        logs[c] = 0

# Calculate Fantasy Points (PPR)
logs['fantasy_points'] = (
    (logs['Passing_Yds'] * 0.04) +
    (logs['Passing_TD'] * 4) +
    (logs['Passing_Int'] * -2) +
    (logs['Rushing_Yds'] * 0.1) +
    (logs['Rushing_TD'] * 6) +
    (logs['Receiving_Rec'] * 1) +
    (logs['Receiving_Yds'] * 0.1) +
    (logs['Receiving_TD'] * 6) -
    (logs['Fumbles_FL'] * 2)
)

logs['clean_name'] = logs['Player'].apply(clean_name)

logs[['Player', 'week', 'fantasy_points']].sort_values('fantasy_points', ascending=False).head()

## 3. Merge & Efficiency Analysis

In [ ]:
merged = pd.merge(logs, financials, on='clean_name', how='inner', suffixes=('_log', '_fin'))

# Calculate Efficiency
# Points per $1M spent that week
# Floor cost to avoid division by zero
merged['weekly_efficiency'] = merged['fantasy_points'] / merged['weekly_cost_m'].clip(lower=0.03)

print(f"Merged Records: {len(merged)}")
merged[['Player', 'week', 'fantasy_points', 'weekly_cost_m', 'weekly_efficiency']].sort_values('weekly_efficiency', ascending=False).head(10)

## 4. Visualize Trends

In [ ]:
# Filter for active players with decent sample
top_players = ['Lamar Jackson', 'Josh Allen', 'Saquon Barkley', 'Justin Jefferson']
subset = merged[merged['Player'].isin(top_players)].sort_values(['Player', 'week'])

plt.figure(figsize=(12, 6))
sns.lineplot(data=subset, x='week', y='weekly_efficiency', hue='Player', marker='o')
plt.title("Weekly Efficiency Trends (2024 Weeks 1-2)")
plt.ylabel("Efficiency (Points / $1M)")
plt.xlabel("Week")
plt.xticks([1, 2]) # Since we only have 2 weeks
plt.legend(title='Player')
plt.show()